# Functional Programming is the Best

## Why are we here?

Functional programming rocks! Everyone is doing it, and it gets cooler every year. 

## Anti-hype

FP is not the be all, end all of it's certainly possible to write a terrible, ugly mess using FP. Your crappy app that no one uses is not going to be better if you rewrite it in Haskell. There's a lot more to programming than some cool ADT tricks and pattern matching. 

Also, this:

![Lisp Comic](http://imgs.xkcd.com/comics/lisp.jpg)

## Anti-anti hype

Learning FP well will make you a better programmer! We have lots of cool stuff happening these days in FP. 

Some examples 

### Ramda 
* 10 times cooler than underscore
* `R.reduce((x,y) => x + y, 0, [1,2,3])`
* Note the pointfree style! Everything in Ramda is autocurrying, too
* `var sum = R.reduce((x,y) => x + y, 0) // [x] => x`

### React
* Ideally, your entire UI is just a function of your original state, and the "virtual DOM" is a purely functional data type that is returned by your component functions and then diff'ed against the last iteration

### Event Sourcing
* Data structures (except for events) theoretically become purely functional
* i.e., `BankAccount(List(AccountOpens(10), Withdrawal(5), Deposit(30), Withdrawal(20))).balance`

### Scala
* We're using a lot of Scala here!
* But is it functional?

## Getting some stuff going here to see if it all works

In [14]:
object HelloWorld {
  def main(args: Array[String]): Unit = {
    println("Hello, world!")
  }
}

defined object HelloWorld

In [15]:
HelloWorld.main(Array())


Hello, world!


In [16]:
object MyModule {
  def abs(n: Int): Int = 
    if (n < 0) -n
    else n

  private def formatAbs(x: Int) = {
    val msg = "The absolute value of %d is %d" 
    msg.format(x, abs(x))
  }

  def main(args: Array[String]): Unit = 
    println(formatAbs(-42))
}

defined object MyModule

In [17]:
MyModule.main(Array())

The absolute value of -42 is 42


## What does this annotation do?
We can see an example of how the `@annotation.tailrec` works here -- the following more naïve implementation won't compile.

In [18]:
def factorial(n: Int): Int = {
    //@annotation.tailrec
    def go(n : Int): Int = 
        if (n == 1) 1
        else n * go(n -1)
    go(n)
}

defined function factorial

In [19]:
factorial(5)

res18: Int = 120

## Fibonacci w/ Tail Recursion

In [20]:
def fib(n: Int): Int = {
    @annotation.tailrec
    def go(nMinus2 : Int, nMinus1 : Int, n: Int, index: Int) : Int = 
        if (n == 0) 1
        else if (n == 1) 1
        else if (index > n) nMinus2 + nMinus1
        else go(nMinus1, nMinus2 + nMinus1, n, index + 1)
    go(1, 1, n, 3)
}

defined function fib

In [21]:
fib(0)
fib(1)
fib(2)
fib(3)
fib(4)
fib(5)
fib(6)
fib(13)

res20_0: Int = 1
res20_1: Int = 1
res20_2: Int = 2
res20_3: Int = 3
res20_4: Int = 5
res20_5: Int = 8
res20_6: Int = 13
res20_7: Int = 377

In [22]:
def isSorted[A](as: Array[A])(ordered: (A,A) => Boolean): Boolean ={
    @annotation.tailrec
    def loop(n: Int): Boolean =
        if (n >= as.length - 1) true 
        else if (!ordered(as(n),as(n+1))) false 
        else loop(n + 1)
    loop(0)
}

defined function isSorted

In [23]:
val tStrings1 = Array("Bob","Jim","Carol")
val tStrings2 = Array("Bob","Dole","Gore")
val tNums1 = Array(3, 2, 1)
val tNums2 = Array(1, 2, 3)
isSorted(tStrings1)(_ < _) // False
/*
isSorted(tStrings2, (x: String, y: String) => x < y) // True
isSorted(tNums1, (x: Int, y: Int) => x < y) // False
isSorted(tNums2, (x: Int, y: Int) => x < y) // True
// Try with a different function
isSorted(tNums2, (x: Int, y: Int) => x > y) // False
// Sanity check some obvious ones
isSorted(Array(), (x: Int, y: Int) => true) // True
isSorted(Array(1), (x: Int, y: Int) => true) // True
*/

tStrings1: Array[String] = Array("Bob", "Jim", "Carol")
tStrings2: Array[String] = Array("Bob", "Dole", "Gore")
tNums1: Array[Int] = Array(3, 2, 1)
tNums2: Array[Int] = Array(1, 2, 3)
res22_4: Boolean = false

## Curry, Uncurry, Compose

In [24]:
def curry[A,B,C](f: (A, B) => C): A => (B => C) = 
    a => b => f(a, b)

defined function curry

In [25]:
val myAdder = (x: Int, y: Int) => x + y
myAdder(2, 3) // 5
val curriedAdder = curry(myAdder)
val twoAdder = curriedAdder(2)
twoAdder(3) // 5
twoAdder(4) // 6

myAdder: (Int, Int) => Int = <function2>
res24_1: Int = 5
curriedAdder: Int => Int => Int = <function1>
twoAdder: Int => Int = <function1>
res24_4: Int = 5
res24_5: Int = 6

In [26]:
def uncurry[A,B,C](f: A => B => C): (A, B) => C = 
    (a, b) => f(a)(b)

defined function uncurry

In [27]:
val uncurriedAdder = uncurry(curriedAdder)
uncurriedAdder(2, 3) // 5

uncurriedAdder: (Int, Int) => Int = <function2>
res26_1: Int = 5

In [28]:
def compose[A,B,C](f: B => C, g: A => B) : A => C = 
    a => f(g(a))

defined function compose

In [29]:
val fiveAdder = compose(twoAdder, curriedAdder(3))
fiveAdder(3) // 8
fiveAdder(15) // 20

fiveAdder: Int => Int = <function1>
res28_1: Int = 8
res28_2: Int = 20

# Functional Data Structures

In [30]:
val sum = (x : List[Int]) => 0
val x = List(1,2,3,4,5) match {
    case x :: 2 :: 4 :: _ => x
    case Nil => 42
    case x :: y :: 3 :: 4 :: _ => x + y 
    case h :: t => h + sum(t)
    case _ => 101
}

sum: List[Int] => Int = <function1>
x: Int = 3

In [31]:
def tail[A](as : List[A]) : List[A] = as match {
    case Nil => Nil
    case a :: as => as
}

defined function tail

In [32]:
tail(List(3, 4, 5))
tail(List())

res31_0: List[Int] = List(4, 5)
res31_1: List[Nothing] = List()

In [33]:
def setHead[A](newHead : A, as: List[A]) = as match {
    case Nil => List(newHead)
    case a :: as => newHead :: as
}

defined function setHead

In [34]:
setHead(7, List(3, 4, 5))

res33: List[Int] = List(7, 4, 5)

In [35]:
def drop[A](l: List[A], n: Int): List[A] = l match {
    case a :: as if n > 0 => drop(as, n - 1)
    case _ => l
}

defined function drop

In [36]:
drop(List(3, 4, 5), 2)
drop(List(), 4)
drop(List(3, 4, 5), 83)

res35_0: List[Int] = List(5)
res35_1: List[Nothing] = List()
res35_2: List[Int] = List()

In [37]:
def dropWhile[A](l: List[A], f: A => Boolean): List[A] = l match {
    case a :: as if (f(a)) => dropWhile(as, f)
    case _ => l
}

defined function dropWhile

In [38]:
dropWhile(List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),(x : Int) => x <= 5)

res37: List[Int] = List(6, 7, 8, 9, 10)

In [39]:
def init[A](l: List[A]): List[A] = l match {
    case _ :: Nil => Nil
    case a :: as => a :: init(as)
}

defined function init

In [40]:
init(List(1, 2, 3, 4))
init(List(1, 2))
init(List(1))

res39_0: List[Int] = List(1, 2, 3)
res39_1: List[Int] = List(1)
res39_2: List[Int] = List()

## Folding

In [41]:
def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B =
 as match {
    case Nil => z
    case x :: xs => f(x, foldRight(xs, z)(f))
 }

defined function foldRight

In [42]:
foldRight(List(1, 2, 3), Nil:List[Int])(_::_)

res41: List[Int] = List(1, 2, 3)

In [43]:
def product(ns: List[Double]) =
    foldRight(ns, 1.0)(_*_)

defined function product

In [44]:
product(List(1, 2, 3))

res43: Double = 6.0

In [45]:
def length[A](as: List[A]): Int = 
    foldRight(as, 0)((_, y) => y + 1)

defined function length

In [46]:
length(List(1, 2, 3, 3932))

res45: Int = 4

In [47]:
def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B = as match {
    case Nil => z
    case a :: as => foldLeft(as, f(z, a))(f)
}

defined function foldLeft

In [48]:
foldLeft(List(1, 2, 3), 1.0)(_*_)

res47: Double = 6.0

In [49]:
def reverse[A](as: List[A]) : List[A] = {
    val switch = (as : List[A], el : A) => el :: as
    foldLeft(as, List[A]())(switch)
}

defined function reverse

In [50]:
reverse(List(1,2,3))

res49: List[Int] = List(3, 2, 1)

In [51]:
def coolFoldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = {
    foldLeft(reverse(as), z)((a, b) => f(b,a))
}

defined function coolFoldRight

In [52]:
def append[A](as: List[A], bs: List[A]) : List[A] = {
    coolFoldRight(as, bs)(_::_)
}

defined function append

In [53]:
append(List(1,2,3),List(4, 5, 6))

res52: List[Int] = List(1, 2, 3, 4, 5, 6)

In [54]:
def concatenate[A](l : List[List[A]]) : List[A] = {
    foldLeft(l, List[A]())(append)
}

defined function concatenate

In [55]:
concatenate(List(List(1, 2, 3), List(4,5,6), List(7,8,9)))

res54: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9)

## More functions for lists

In [56]:
def addOneToAll(xs : List[Int]) : List[Int] = xs match {
    case Nil => Nil
    case x :: xs => x + 1 :: addOneToAll(xs)
}

defined function addOneToAll

In [57]:
addOneToAll(List(1,2,3))

res56: List[Int] = List(2, 3, 4)

In [58]:
def allDoublesToString(xs: List[Double]) : List[String] = xs match {
    case Nil => Nil
    case x :: xs => x.toString() :: allDoublesToString(xs)
}

defined function allDoublesToString

In [59]:
allDoublesToString(List(3.3, 3.2, 4.2))

res58: List[String] = List("3.3", "3.2", "4.2")

In [60]:
def map[A,B](as: List[A])(f: A => B): List[B] = as match {
    case Nil => Nil
    case x :: xs => f(x) :: map(xs)(f)
}

defined function map

In [61]:
map(List(1,2,3))(_+1)

res60: List[Int] = List(2, 3, 4)

In [62]:
def filter[A](as: List[A])(f: A => Boolean): List[A] = as match {
    case Nil => Nil
    case x :: xs if f(x) => x :: filter(xs)(f)
    case x :: xs => filter(xs)(f)
}

defined function filter

In [63]:
filter(List(1,2,3,4,5,6,7,8,9))(_ % 2 == 0)

res62: List[Int] = List(2, 4, 6, 8)

In [64]:
def flatMap[A,B](as: List[A])(f: A => List[B]): List[B] = as match {
    case Nil => Nil
    case x :: xs => append(f(x), flatMap(xs)(f))
}

defined function flatMap

In [65]:
flatMap(List(1,2,3))(i => List(i,i))

res64: List[Int] = List(1, 1, 2, 2, 3, 3)

In [66]:
def filterFlatMap[A](as: List[A])(f: A => Boolean): List[A] = {
    val filterFunc = (x : A) => if (f(x)) List(x) else List[A]()
    flatMap(as)(filterFunc)
}

defined function filterFlatMap

In [67]:
filterFlatMap(List(1,2,3,4,5,6,7,8,9))(_ % 2 == 0)

res66: List[Int] = List(2, 4, 6, 8)

In [68]:
def addTwoLists(xs : List[Int], ys : List[Int]) : List[Int] = (xs,ys) match {
    case (Nil, Nil) => Nil
    case (x :: xs, y :: ys) => x + y :: addTwoLists(xs, ys)
}

defined function addTwoLists

In [69]:
addTwoLists(List(1,2,3),List(4,5,6))

res68: List[Int] = List(5, 7, 9)

In [70]:
def zipWith[A,B,C](xs : List[A], ys : List[B])(f: (A,B) => C) : List[C] = (xs,ys) match {
    case (Nil, Nil) => Nil
    case (x :: xs, y :: ys) => f(x,y) :: zipWith(xs, ys)(f)
}

defined function zipWith

In [71]:
zipWith(List(1,2,3),List(4,5,6))(_+_)

res70: List[Int] = List(5, 7, 9)

In [72]:
def hasSubsequence[A](sup: List[A], sub: List[A]): Boolean = (sup, sub) match {
    case (_, Nil) => true
    case (Nil, sub) => false
    case (x :: sup, y :: sub) if (x == y) => hasSubsequence(sup, sub)
    case (x :: sup, y :: sub) => hasSubsequence(sup, y :: sub)
}

defined function hasSubsequence

In [73]:
hasSubsequence(List(1,2,3,4), List(3,4))
hasSubsequence(List(1,2,3,4), List(4,3))

res72_0: Boolean = true
res72_1: Boolean = false

## Trees

In [74]:
sealed trait Tree[+A]
case class Leaf[A](value: A) extends Tree[A]
case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

defined trait Tree
defined class Leaf
defined class Branch

In [75]:
val myTree = Branch(Branch(Leaf("A"), Leaf("B")), Branch(Leaf("C"), Leaf("D")))

myTree: Branch[String] = Branch(Branch(Leaf(A),Leaf(B)),Branch(Leaf(C),Leaf(D)))

In [76]:
def size[A](t : Tree[A]) : Int = t match {
   case Leaf(_) => 1
   case Branch(left, right) => 1 + size(left) + size(right)
}

defined function size

In [77]:
size(myTree)

res76: Int = 7

In [78]:
val intTree = Branch(Branch(Leaf(1), Leaf(2)), Branch(Branch(Leaf(5),Leaf(7)), Leaf(6)))

intTree: Branch[Int] = Branch(Branch(Leaf(1),Leaf(2)),Branch(Branch(Leaf(5),Leaf(7)),Leaf(6)))

In [79]:
def maximum(t : Tree[Int]) : Int = t match {
    case Leaf(x) => x
    case Branch(left, right) => maximum(left) max maximum(right)
}

defined function maximum

In [80]:
maximum(intTree)

res79: Int = 7

In [81]:
def depth[A](t : Tree[A]) : Int = t match {
    case Leaf(_) => 1
    case Branch(left, right) => 1 + (depth(left) max depth(right))
}

defined function depth

In [82]:
depth(intTree)

res81: Int = 4

In [83]:
def treeMap[A,B](t : Tree[A])(f: A => B) : Tree[B] = t match {
    case Leaf(x) => Leaf(f(x))
    case Branch(left, right) => Branch(treeMap(left)(f), treeMap(right)(f))
}

defined function treeMap

In [84]:
treeMap(intTree)(_+1)

res83: Tree[Int] = Branch(Branch(Leaf(2),Leaf(3)),Branch(Branch(Leaf(6),Leaf(8)),Leaf(7)))

In [85]:
def fold[A,B](t : Tree[A])(leafF : A => B)(branchF : (B,B) => B) : B = t match {
    case Leaf(x) => leafF(x)
    case Branch(left, right) => branchF(fold(left)(leafF)(branchF), fold(right)(leafF)(branchF))
}

defined function fold

In [86]:
def fsize[A](t : Tree[A]) : Int = 
   fold(t)(x => 1)(_ + _ + 1)
fsize(myTree)

defined function fsize
res85_1: Int = 7

In [87]:
def fmaximum(t : Tree[Int]) : Int = 
    fold(t)(x => x)(_ max _)

defined function fmaximum

In [88]:
fmaximum(intTree)

res87: Int = 7

In [90]:
def fdepth[A](t : Tree[A]) : Int = 
    fold(t)(x => 1)((_ max _) + 1)

: 

In [90]:
fdepth(intTree)

res89: Int = 3